<a href="https://colab.research.google.com/github/burnedsyn/enablecudaopencvcolab/blob/main/PRJ_VIDEO_mesh_cr%C3%A9ation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/opencv/opencv
!git clone https://github.com/opencv/opencv_contrib
!mkdir /content/build
%cd /content/build

!cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules  \
       -DBUILD_SHARED_LIBS=OFF \
       -DBUILD_TESTS=OFF \
       -DBUILD_PERF_TESTS=OFF \
       -DBUILD_EXAMPLES=OFF \
       -DWITH_OPENEXR=OFF \
       -DWITH_CUDA=ON \
       -DWITH_CUBLAS=ON \
       -DWITH_CUDNN=ON \
       -DOPENCV_DNN_CUDA=ON \
       /content/opencv

!make -j8

In [ ]:
# connect your Google Drive
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
## save for later use:
%cd /content/drive/MyDrive
!mkdir cv2_cuda
!cp  /content/build/lib/python3/cv2.cpython-37m-x86_64-linux-gnu.so   "/content/drive/MyDrive/cv2_cuda/"



Accelerateur gpu

In [ ]:
import cv2
import mediapipe as mp
import tensorflow as tf

# Enable TensorFlow GPU support
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Using GPU: ", tf.test.gpu_device_name())
    except RuntimeError as e:
        print(e)

# Create the Face Mesh model
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Create a VideoCapture object to read the video file
cap = cv2.VideoCapture('/content/gdrive/MyDrive/AlicePRJ/alicvideo.mp4')


# Read the first frame to get the frame size
ret, frame = cap.read()
frame_height, frame_width, _ = frame.shape

# Create a CUDA stream for GPU-accelerated OpenCV functions
cuda_stream = cv2.cuda_Stream()

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create CUDA matrices for GPU-accelerated OpenCV functions
    cuda_image = cv2.cuda_GpuMat(image, allocator=cv2.cuda.HostMemAllocator())
    cuda_gray = cv2.cuda_GpuMat((frame_height, frame_width), cv2.CV_8UC1, allocator=cv2.cuda.HostMemAllocator())
    cuda_threshold = cv2.cuda_GpuMat((frame_height, frame_width), cv2.CV_8UC1, allocator=cv2.cuda.HostMemAllocator())

    # Convert the image to grayscale on the GPU
    cv2.cuda.cvtColor(cuda_image, cv2.COLOR_RGB2GRAY, dst=cuda_gray, stream=cuda_stream)

    # Threshold the grayscale image on the GPU
    cv2.cuda.threshold(cuda_gray, 127, 255, cv2.THRESH_BINARY, dst=cuda_threshold, stream=cuda_stream)

# Download the thresholded image back to the CPU
thresholded_image = cuda_threshold.download()

# Process the thresholded image with the Face Mesh model
results = face_mesh.process(thresholded_image)

# Draw the face mesh on the frame
if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

# Display the frame
cv2.imshow('Frame', frame)

# Wait for a key press
if cv2.waitKey(1) & 0xFF == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()
